#       REPORT for Part A: Linear Regression

The boston house prices datasets is used to predict the prices of new homes on the market. With this machine language structure, linear and ridge regression were implemeneted to get a prediction. The structure utilizes both of these models with exhaustive testing of the dataset through a k-fold validaton of k spits. For each unique k split, only one fold is the tester and the other trainers. These unique data sets are used on the liner and reidge regressions for each iteration of k different folds you will get. For the ridge regression, we also iterate based on the different values of lambda(alpha) that is used for the k-fold validation. The total error averages for each linear regression and ridge regression for each alpha are determined with the overall minimum used as the model for predicting housing prices.  

###### Importing the various needed python libraries and boston data set that will used to train/test the model.

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.datasets import load_boston######
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [71]:
bostonh_data = load_boston()

###### Assigning references to different areas of the dataset. X holds the data and y holds its targets. The y variable is also reshaped into a column vector. 

In [72]:
# Create X and Y variables - X holding the .data and Y holding .target 
X = bostonh_data.data
y = bostonh_data.target

#  Reshape Y to be a rank 2 matrix using y.reshape()
y = y.reshape(X.shape[0], 1)

# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', bostonh_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
# Observing the first 2 rows of the data
print(X[0:2])

The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


###### Creating a polynomial of degree 2 and fitting the X data to that polynomial and printing the new shape of both X and y 

In [66]:
# Create a PolynomialFeatures object with degree = 2. Using PolynomialFeatures(degree=2)
# Transform X and save it into X_2 using poly.fit_transform(X)
# Simply copy Y into Y_2 

poly = PolynomialFeatures(degree=2)
X_2 = poly.fit_transform(X)
y_2 = y.copy()

In [10]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


###   Defining the functions of linear and ridge regression

Linear regression functon using normal function gets theat and avoids using gradients . This is done by multiplying a matrix X with its tranpose, getin the inverse and multiplying it by its orginal. 

Ridge regression involves the use of gradient to get theta which entails multiplying a identity matrix the same size as X by a set of alphas ranging from 10^1, 10^1.5,....,10^7. The scaled matrix is then added to the product of X and its transpose. This is then inverted and multplied with the oproduct of X and it transpose


In [ ]:
# Define the get_coeff_linear_normaleq function. Use the normal equation method.
# Return w values
#Using the equation from the Project Lab slides
#  w = (x^T X)^-1 (X^T y)
def get_coeff_linear_normaleq(X_train, y_train):
    w = np.matmul(np.linalg.pinv(np.matmul(X_train.T, X_train)), np.matmul(X_train.T, y_train))
    return w

In [85]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values
#Using the equation from the Project Lab slides
#w = ((x^T X)+ aI)^-1(X^T y)
#term1 = (x^T X)
#term1 = (aI)
#term1 = (x^T y)

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    term1 = np.matmul(X_train.T, X_train)
    term2 = np.identity(X_train.shape[1])*alpha
    term3 = np.matmul(X_train.T, y_train)
    w = np.matmul(np.linalg.pinv(term1+term2), term3)
    return w

###    Evaluating the errors 

The errors of the predicted values for housing prices are obtaineed from running model with training data and getting the difference between the predicted values and the test data. The predicted values are obtained from  multplying X with either regression. This difeerence is squared and and all the values are averaged. 

###    Implementing the k fold cross validation 

Which computes the linear or ridge regression of the data set for each varying k fold to get an exhaustive testing of the model. It splits the data sets into a set number of k folds where only one fold is the tester at a time, this is changed with each iteration. The data is centered and scaled, and depending on choice whichever regression is used to obtain theta and its error calculated for each iteration. This error is then taken into an overall average. 

In [80]:
# Define the evaluate_err_ridge function.
# Return the train_error and test_error values
# Using the equation from the lecture 3 slides 

def evaluate_err(X_train, X_test, y_train, y_test, w): 
#     pred_train=prediction using w and X_tran+np.mean(y_train)
#     pred_test=prediction using w and X_test
#     remember to add the mean back
#     train_error=...
#     test_error=...

    #getting a matrix that consists of all the n predicted values of y train and y test
    pred_train = np.matmul(X_train, w) + np.mean(y_train) 
    pred_test = np.matmul(X_test, w) + np.mean(y_train)
    
    #getting matrces of the diffrence between all n values of y test and y train and their respective predicted values squared
    train_error = (y_train - pred_train)**2
    test_error = (y_test - pred_test) **2
    
    #getting the mean square error from the summation of all the diferecnes in value divided by the total number of elements
    train_error = np.sum(train_error)/y_train.shape[0]
    test_error = np.sum(test_error)/y_test.shape[0]
    
    return train_error, test_error

In [90]:
# Finish writting the k_fold_cross_validation function. 
# Returns the average training error and average test error from the k-fold cross validation
# Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha, linrid):
    min_train_error = 9999
    min_test_error = 9999
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        y_train_mean = np.mean(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        
        # Scaling the data matrix
        scaler=preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Determine the training error and the test error
        
        #Derterming whether to use linear or ridge regression, linear if true, ridge otherwise
        if linrid == True:
            w = get_coeff_linear_normaleq(X_train, y_train)
        else:
            w = get_coeff_ridge_normaleq(X_train, y_train, alpha)
     
        min_train_error, min_test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        
        #getting the total performance for the regression determinded by the k fold validation
        total_E_val_test += min_test_error
        total_E_val_train += min_train_error
        
    #averaging the performance of the regressioon for the overall k folds
    total_E_val_test = total_E_val_test/k
    total_E_val_train = total_E_val_train/k
    
    return  total_E_val_test, total_E_val_train

###    Finding the best lambda

Using any number for k to represent folds, the variation was conducted for each alpha to get its total average error and then the alpha with the minimm error was declared the best one. This model used th eoriginal X and y data

The model was then ran with the X^2 polynomial data and the best alpha was found for that set too 

In [102]:
alpha = np.logspace(1, 7, num=13)
k = 10
min_train_error = 9999
min_test_error = 9999

total_error_test, total_error_train = k_fold_cross_validation(k, X, y, alpha, True)
print("########LINEAR REGRESSION############")
print ("Total test error: ", total_error_test)
print ("Total train error: ", total_error_train)
print("_____________________________________________________\n\n") 

print("########RIDGE REGRESSION############")
for x in range(0, 13):
    total_error_test, total_error_train = k_fold_cross_validation(k, X, y, alpha[x], False)
    print("_____________________________________________________")
    print("Alpha: ", alpha[x])
    print("Test error: ", total_error_test)
    print("Train error: ", total_error_train)
    
    #determines the overall minimum alpha throughout the k-fold validation
    if total_error_test < min_test_error:
        min_test_error = total_error_test
        min_train_error = total_error_train
        min_alpha = alpha[x]

print("\n\n")
print ("Best alpha: ", min_alpha)
print ("Minimum test error: ", min_test_error)
print ("Minimum train error: ", min_train_error)

########LINEAR REGRESSION############
Total test error:  23.773613682085497
Total train error:  21.885480719855178
_____________________________________________________


########RIDGE REGRESSION############
_____________________________________________________
Alpha:  10.0
Test error:  23.7890592502954
Train error:  21.959504864528657
_____________________________________________________
Alpha:  31.622776601683793
Test error:  24.03950189103493
Train error:  22.30713860896172
_____________________________________________________
Alpha:  100.0
Test error:  25.130684396920145
Train error:  23.610271468580894
_____________________________________________________
Alpha:  316.22776601683796
Test error:  28.88537110197723
Train error:  27.674191548972622
_____________________________________________________
Alpha:  1000.0
Test error:  38.41394839825479
Train error:  37.49236322124653
_____________________________________________________
Alpha:  3162.2776601683795
Test error:  53.37145474383

In [103]:
total_error_test, total_error_train = k_fold_cross_validation(k, X_2, y_2, alpha, True)
print("########LINEAR REGRESSION############")
print ("Total test error: ", total_error_test)
print ("Total train error: ", total_error_train)
print("_____________________________________________________\n\n") 

print("########RIDGE REGRESSION############")
for x in range(0, 13):
    total_error_test, total_error_train = k_fold_cross_validation(k, X_2, y_2, alpha[x], False)
    print("_____________________________________________________")
    print("Alpha: ", alpha[x])
    print("Test error: ", total_error_test)
    print("Train error: ", total_error_train)
    if total_error_test < min_test_error:
        min_test_error = total_error_test
        min_train_error = total_error_train
        min_alpha = alpha[x]

print("\n\n")
print ("Best alpha: ", min_alpha)
print ("Minimum test error: ", min_test_error)
print ("Minimum train error: ", min_train_error)

########LINEAR REGRESSION############
Total test error:  12.688085050328374
Total train error:  5.974831823338861
_____________________________________________________


########RIDGE REGRESSION############
_____________________________________________________
Alpha:  10.0
Test error:  13.404801243846377
Train error:  10.024584491538686
_____________________________________________________
Alpha:  31.622776601683793
Test error:  15.715600046721931
Train error:  12.617122959383908
_____________________________________________________
Alpha:  100.0
Test error:  18.885442276702758
Train error:  16.065010589998312
_____________________________________________________
Alpha:  316.22776601683796
Test error:  21.91738957056631
Train error:  19.52011837138036
_____________________________________________________
Alpha:  1000.0
Test error:  25.771733205654787
Train error:  23.900948880551617
_____________________________________________________
Alpha:  3162.2776601683795
Test error:  33.6249281

# Conclusion

The best lamba is 10 when using the X data set transformed to fit a polynomial of degree 2 as it results in the lowest minimum erorr for both the teting and trainng data splits 